In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from chapter2 import conf
from chapter2.data import make_dataset

%reload_ext autoreload
%aimport chapter2.conf
%aimport chapter2.data.make_dataset
%autoreload 1

In [3]:
housing = make_dataset.load_data(
    conf.ROOT_DATA_DIR, conf.DataDirectory.INTERIM, "housing_train_prepared.csv"
)

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection

y = housing["median_house_value"]
X = housing.drop("median_house_value", axis=1)
rf = RandomForestRegressor(max_depth=20, random_state=0, n_jobs=-1)
X_train, X_val, y_train, y_val = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=0
)
rf.fit(X_train, y_train)
rf.score(X_val, y_val)

0.7762066750709506

In [13]:
n_estimators = [5, 20, 50, 100]  # number of trees in the random forest
max_features = [
    "auto",
    "sqrt",
    0.8,
]  # number of features in consideration at every split
max_depth = [10, 30, 90]  # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10]  # minimum sample number to split a node
min_samples_leaf = [1, 3, 10]  # minimum sample number that can be stored in a leaf node
bootstrap = [True, False]  # method used to sample data points

random_grid = {
    "n_estimators": n_estimators,
    "max_features": max_features,
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
    "min_samples_leaf": min_samples_leaf,
    "bootstrap": bootstrap,
}

possibilities = 1
for param in random_grid.values():
    possibilities = possibilities * len(param)
print(f"{possibilities} parameter possibilities")

648 parameter possibilities


In [14]:
from sklearn.model_selection import RandomizedSearchCV

rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=100,
    cv=5,
    random_state=0,
    n_jobs=-1,
)

In [15]:
rf_random.fit(X, y)

RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(max_depth=20, n_jobs=-1,
                                                   random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 30, 90],
                                        'max_features': ['auto', 'sqrt', 0.8],
                                        'min_samples_leaf': [1, 3, 10],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': [5, 20, 50, 100]},
                   random_state=0)

In [10]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 3,
 'max_features': 0.8,
 'max_depth': 90,
 'bootstrap': True}

In [11]:
rf_random.best_score_

0.7729116716722273

In [20]:
rf_random.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_estimators', 'param_min_samples_split', 'param_min_samples_leaf', 'param_max_features', 'param_max_depth', 'param_bootstrap', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [27]:
results_df = pd.DataFrame(rf_random.cv_results_)

In [28]:
for param in random_grid:
    results_df[param].sort_values()

KeyError: 'n_estimators'